In [1]:
from torch import nn
import torch
from torch.utils.tensorboard import SummaryWriter
import gym
from torch.distributions import Normal
from gym.spaces import Box
import random
from IPython.display import clear_output
from torch.distributions import MultivariateNormal
import sys
sys.path.append('C:\\Users\\Максимилиан\\Desktop\\Skoltech\\Reinforcement learning\\Final project')

In [2]:
from casadi import *
%matplotlib widget
import matplotlib.pyplot as plt
#%matplotlib inline
from gekko import GEKKO
from Dynamics import get_next_state, state_to_coords, get_energy
from Environment import DoublePendulumEnv, normalize_angle
from PPO.Proximal_Policy_Optimization import PPO, unscaled_action
from PPO.train import train

# MPC based approach

In [3]:
!pip install do-mpc

In [4]:
"""
Use MPC to control the Pendulum.
In this version an additional real model is needed for simulation.
"""


# simulate and choose the best action
def choose_action(state, rollout, horizon):
    """
    :param rollout: trajectories that are simulated.
    :param horizon: steps lookahead.
    :return:
    """
    best_action = None
    max_trajectory_value = -float('inf')
    for trajectory in range(rollout):
        env.state = state
        trajectory_value = 0
        for h in range(horizon):
            action = np.array([random.uniform(-5, 5)])
            if h == 0:
                first_action = action
            _, reward, _, _ = env.step(action)
            trajectory_value += reward
        # check if this trajectory's value is higher.
        if trajectory_value > max_trajectory_value:
            max_trajectory_value = trajectory_value
            best_action = first_action
    return best_action

In [5]:
def start():
    for rollout, horizon in zip(rollout_list, horizon_list):
        episode_reward_list = []
        for episode in range(max_episodes):
            state = env.reset()
            episode_reward = 0
            for step in range(max_episode_steps):
#                 if render:
#                     env.render()
                state = env.state
                action = choose_action(state, rollout, horizon)
                next_state, reward, done, info = env.step(action)
                episode_reward += reward
            print('episode %d ends with reward %d' % (episode, episode_reward))
            episode_reward_list.append(episode_reward)
        plt.plot(episode_reward_list, label='rollout=%d horizon=%d' % (rollout, horizon))

In [ ]:
state = [0, np.pi/2, np.pi/2, 0, 0, 0]
if __name__ == '__main__':
    seed = 1000
    max_episodes = 10000
    max_episode_steps = 500
    env = DoublePendulumEnv(init_state = state )
    
    state = env.reset()
    
    np.random.seed(seed)
    random.seed(seed)
    render = True

    # custom the rollout number and horizon number
    rollout_list = [50]
    horizon_list = [10]
    start()

    env.close()
    plt.legend()
    plt.grid()
    plt.xlabel('episode')
    plt.ylabel('episode reward')
    plt.title('MPC_Pendulum_v0')
    # plt.savefig('Naive_MPC/MPC_Pendulum_v0.png')

Environment initialized
